In [1]:
!pip install matlab

In [6]:
cd (fullfile(matlabroot,'extern','engines','python'))
system('python setup.py install')

NameError: name 'cd' is not defined

In [5]:
% Radar parameters
fc = 77e9;             % Carrier frequency (Hz)
fs = 1e6;              % Sampling rate (Hz)
c = physconst('LightSpeed'); % Speed of light (m/s)
pulseWidth = 1e-6;     % Pulse width (s)
prf = 1e3;             % Pulse repetition frequency (Hz)
rangeMax = 500;        % Maximum unambiguous range (m)

% Define the radar system
waveform = phased.RectangularWaveform('PulseWidth', pulseWidth, ...
    'PRF', prf, 'SampleRate', fs);
transmitter = phased.Transmitter('PeakPower', 1e3, 'Gain', 20);
receiver = phased.ReceiverPreamp('Gain', 40, 'NoiseFigure', 5, ...
    'SampleRate', fs);

% Define the target
target = phased.RadarTarget('MeanRCS', 1, 'OperatingFrequency', fc);
targetPlatform = phased.Platform('InitialPosition', [100; 0; 0], ...
    'Velocity', [30; 0; 0]);

% Define the radar platform
radarPlatform = phased.Platform('InitialPosition', [0; 0; 0], ...
    'Velocity', [0; 0; 0]);

% Create a radar signal collector
collector = phased.Collector('Sensor', phased.IsotropicAntennaElement, ...
    'OperatingFrequency', fc);

% Create a free space channel
channel = phased.FreeSpace('SampleRate', fs, 'TwoWayPropagation', true, ...
    'OperatingFrequency', fc);

% Simulate one pulse
txsig = waveform();
txsig = transmitter(txsig);

% Propagate the signal to the target
[tgtpos, tgtvel] = targetPlatform(1/fs);
[radpos, radvel] = radarPlatform(1/fs);
txsig = channel(txsig, radpos, tgtpos, radvel, tgtvel);

% Reflect the signal off the target
txsig = target(txsig);

% Propagate the signal back to the radar
txsig = channel(txsig, tgtpos, radpos, tgtvel, radvel);

% Receive the signal
rxsig = receiver(txsig);

% Collect the signal
rxsig = collector(rxsig, radpos);

% Plot the received signal
t = (0:length(rxsig)-1)/fs;
figure;
plot(t*1e6, abs(rxsig));
xlabel('Time (\mus)');
ylabel('Amplitude');
title('Received Radar Signal');

% Matched filter to improve SNR
mf = phased.MatchedFilter('Coefficients', getMatchedFilter(waveform));
rxsig_filt = mf(rxsig);
t_filt = unigrid(0, size(rxsig_filt, 1)-1, size(rxsig_filt, 1), 'column')/fs;

% Detect peaks
threshold = 0.5 * max(abs(rxsig_filt));
detected = abs(rxsig_filt) > threshold;

% Estimate range
[~, rangeBins] = findpeaks(abs(rxsig_filt), 'MinPeakHeight', threshold);
range = rangeBins * (c / (2 * fs));

% Display detected range
disp(['Detected range: ', num2str(range), ' meters']);

% Save the radar data
save('radar_data.mat', 'rxsig', 't', 'range');


SyntaxError: invalid syntax (2576322855.py, line 2)